In [0]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;


In [2]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [5]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision


tcmalloc: large alloc 1073750016 bytes == 0x585e0000 @  0x7f0398ee02a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

from torchvision import datasets, transforms
from torch.autograd import Variable

In [7]:
kwargs = {}
train_data = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                    transform=transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize((.1307,),(.3081,))])),
batch_size=64, shuffle=True, **kwargs)
    
test_data = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, 
                    transform=transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize((.1307,),(.3081,))])),
batch_size=64, shuffle=True, **kwargs)

Processing...
Done!


#ResNet implemenation

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=1,
                     bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes,
                     out_planes,
                     kernel_size = 1,
                     stride = stride,
                     bias=False)

In [0]:
class BasicBlock(nn.Module):
    expansion = 1
    
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
        
        
    def forward(self, x):
        residual = x # save x
        
        # conv -> bn -> relu
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            residual = self.downsample(x)
        
        # x + F(x) - this realizes the shortcut conn.
        out += residual
        out = self.relu(out) # final relu
        
        return out

In [0]:
class Bottleneck(nn.Module):
    """
    The expansion factor controls the number of output
    channels of the last 1x1 convolution layer.
    """
    expansion = 4
    
    def __init__(self, inplanes, planes, stride = 1, downsample=None):
        super(Bottleneck, self).__init__()
        
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes*self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        residual = x
    
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.downsample is not None:
            residual = self.downsample(x)
            
        out += residual
        out = self.relu(out)
        
        return out

In [0]:
#from torchvision.models import resnet18

class ResNet(nn.Module):
    
    def __init__(self, block, layers, num_classes=10):
        self.inplanes = 10
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 2, stride = 1, padding = 0,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(10)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=1, padding=0)
        self.layer1 = self._make_layer(block, 10, layers[0])
        self.layer2 = self._make_layer(block, 20, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 40, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 80, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(3, stride=1)
        self.fc = nn.Linear(320 * block.expansion, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        #print(x.size())
        #exit()
        x = x.view(x.size(0), -1)
        x = self.fc(x)      

        return x

In [0]:
kwargs = {}

def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model

#Run the ResNet Model (Error)

In [33]:
device = 'cuda'

model = resnet18()

opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

model.to(device)
model.train()

n_epochs = 30
losses = []

for i in range(n_epochs):
    e_loss = 0
    
    for x,y in train_data:
        
        x = x.to(device)
        y = y.to(device)
        
        model.zero_grad()    # zero gradients
        out = model(x)       # forward pass
        loss = criterion(out, y)  # compute loss
        e_loss += loss.item()     # track loss (optional)
        loss.backward()           # compute gradients via backprop.
        opt.step()                # take an optimizer step
        
    if i % 1 == 0:
      losses.append(e_loss)

    print('Epoch {}: {:.4f}'.format(i, e_loss))

Epoch 0: 180.6076
Epoch 1: 38.7938
Epoch 2: 25.7098
Epoch 3: 16.5908
Epoch 4: 10.7210
Epoch 5: 8.0063
Epoch 6: 5.4929
Epoch 7: 5.2565
Epoch 8: 4.1016
Epoch 9: 1.8574
Epoch 10: 1.2188
Epoch 11: 1.2710
Epoch 12: 0.5319
Epoch 13: 0.4222
Epoch 14: 0.3431
Epoch 15: 0.2189
Epoch 16: 0.2024
Epoch 17: 0.1608
Epoch 18: 0.1328
Epoch 19: 0.1019
Epoch 20: 0.1115
Epoch 21: 0.0954
Epoch 22: 0.0981
Epoch 23: 0.0985
Epoch 24: 0.0903
Epoch 25: 0.0844
Epoch 26: 0.0644
Epoch 27: 0.0829
Epoch 28: 0.0701
Epoch 29: 0.0611


In [0]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(5,3))
plt.plot(e_loss)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training loss'])

In [0]:
def evaluate(model, loader):
    model.eval()
    running_corrects = 0
    for x,y in loader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == y.data)

    print("Accuracy: ", float(running_corrects)/(len(test_data)*64)*100.0)

In [37]:
evaluate(model, test_data)

Accuracy:  99.0047770700637
